# Monte Carlo-GCN(MCGCN) 
For:

Learning from the Dark: Boosting Graph Convolutional Neural Networks wit Diverse Negative Samples.

This code can be run directly on colab

Getting negative samples **based on Monte Carlo chains** refer to this article:

[Graph Convolutional Neural Networks for Web-Scale Recommender Systems](https://arxiv.org/pdf/1806.01973.pdf)

# 0 Preparing

In [ ]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install -q torch-geometric

import torch
import networkx as nx
import matplotlib.pyplot as plt

     |████████████████████████████████| 3.0 MB 33.8 MB/s 
     |████████████████████████████████| 1.6 MB 27.5 MB/s 
     |████████████████████████████████| 222 kB 30.0 MB/s 
     |████████████████████████████████| 376 kB 53.7 MB/s 
     |████████████████████████████████| 45 kB 3.4 MB/s 


## 0.1 Functions

In [ ]:
from collections import defaultdict
import numpy as np
def load_item_pop(X_train):
    item_pop = list()
    node_deg = dict()
    dd = defaultdict(list)
    #edge: <node1> <node2>
    for edge in X_train:
        #   <node2>       <node1> 
        dd[int(edge[1])].append(int(edge[0]))
    for key in dd.keys():
        item_pop.append(1)
    #deg_sum : nodes number
    deg_sum = np.sum(item_pop)
    for key in dd.keys():
        node_deg[key] = 1/deg_sum
    return node_deg, dd

### 0.1.1 Get DFS path for each node

In [ ]:
import time
from collections import defaultdict

#Get DFS path for each node
class Personalized():
    def __init__(self, nx_G, mask, walks_num):
        self.G = nx_G
        self.mask = mask
        self.walks_num = walks_num

    # walks_num = 5
    def dfs(self, start_node):
        stack=[]
        stack.append(start_node)
        seen=set()
        seen.add(start_node)
        walks = []
        mask_list = set(self.mask[start_node])
        while (len(stack)>0):
            vertex=stack.pop()
            nodes=self.G[vertex]
            for w in nodes:
                if w not in seen:
                    stack.append(w)
                    seen.add(w)
            # Skip first-order nearest neighbors
            if vertex in mask_list:
                pass
            else:
                walks.append(vertex)
            if len(walks) >= self.walks_num:
                break
        return walks
    
    def intermediate(self):
        i = 0 
        candidate = defaultdict(list)
        for node in self.G.nodes():
            walk = self.dfs(node)
            candidate[node].extend(walk)
        return candidate

#DFS sequence for each node saved in candidates
def candidate_choose(nx_Graph, mask, walks_num):
    G = Personalized(nx_Graph, mask, walks_num)
    candidates = G.intermediate()
    return candidates

### 0.1.2 Basic GCN MessagePassing Layer

In [ ]:
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
from torch_scatter import gather_csr, scatter
class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='add')  # "Add" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)

        # Step 3-5: Start propagating messages.
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j, edge_index, size):
        # x_j has shape [E, out_channels]
        # edge_index has shape [2, E]

        # Step 3: Normalize node features.
        row, col = edge_index
        deg = degree(row, size[0], dtype=x_j.dtype)  # [N, ]
        deg_inv_sqrt = deg.pow(-0.5)   # [N, ]
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        return norm.view(-1, 1) * x_j
            
    def update(self, aggr_out):
        # aggr_out has shape [N, out_channels]

        # Step 5: Return new node embeddings.
        return aggr_out

# 1. Load datasets

In [ ]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.datasets import KarateClub

# dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())
dataset = Planetoid(root='data/Planetoid', name='Citeseer', transform=NormalizeFeatures())
# dataset = Planetoid(root='data/Planetoid', name='Pubmed')


print()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
print(f'Contains self-loops: {data.contains_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Processing...
Done!

Dataset: Citeseer():
Number of graphs: 1
Number of features: 3703
Number of classes: 6

Data(edge_index=[2, 9104], test_mask=[3327], train_mask=[3327], val_mask=[3327], x=[3327, 3703], y=[3327])
Number of nodes: 3327
Number of edges: 9104
Average node degree: 2.74
Number of training nodes: 120
Training node label rate: 0.04
Contains isolated nodes: True
Contains self-loops: False
Is undirected: True


## 1.1 Calculating the maximum connected subgraph

In [ ]:
from torch_geometric.utils import to_networkx, from_networkx
import networkx as nx

# transform data from torch_geometric to networkx
G = to_networkx(data, to_undirected=True)
# Get the index of all points of the max-connected subgraph
Subnode = max(nx.connected_components(G), key=len)
print(len(Subnode))
# Get the subgraph
SubGraph = G.subgraph(Subnode)
SubnodeList = list(Subnode)
print(SubnodeList)

2120
[1, 5, 8, 10, 12, 13, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 38, 39, 40, 42, 43, 44, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 61, 62, 65, 66, 69, 70, 72, 75, 76, 77, 78, 79, 80, 81, 83, 84, 87, 88, 90, 91, 92, 93, 95, 96, 98, 99, 100, 101, 103, 104, 105, 106, 107, 110, 113, 114, 115, 118, 119, 122, 123, 124, 126, 128, 130, 131, 132, 134, 135, 136, 137, 138, 142, 144, 147, 148, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 162, 167, 168, 169, 170, 172, 173, 177, 178, 180, 181, 184, 186, 188, 189, 190, 191, 194, 195, 197, 198, 200, 201, 203, 204, 205, 206, 208, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 224, 226, 227, 228, 229, 230, 231, 232, 234, 236, 237, 240, 241, 242, 243, 244, 246, 247, 249, 250, 252, 253, 254, 255, 258, 259, 260, 263, 265, 266, 267, 268, 269, 272, 273, 274, 280, 285, 286, 287, 289, 292, 293, 294, 298, 300, 302, 303, 304, 307, 308, 311, 312, 313, 314, 316, 317, 318, 319, 321, 322, 325, 330, 331, 332, 333,

## 1.2 Transform data from networkx to torch_geometric

In [ ]:
PyTSubGraph = from_networkx(SubGraph)
SubGraph = to_networkx(PyTSubGraph, to_undirected=True)

PyTSubGraph.test_mask = data.test_mask[SubnodeList]
PyTSubGraph.train_mask = data.train_mask[SubnodeList]
PyTSubGraph.val_mask = data.val_mask[SubnodeList]
PyTSubGraph.x = data.x[SubnodeList]
PyTSubGraph.y = data.y[SubnodeList]

# Get number of nodes
num_nodes = PyTSubGraph.num_nodes
# Get edge_index
edge_index = PyTSubGraph.edge_index
edge_value = torch.ones(edge_index.size(1), device=edge_index.device)
# Transforming the adjacency matrix from a sparse transpose matrix to a dense matrix
adjdense = torch.sparse.FloatTensor(edge_index, edge_value, torch.Size([num_nodes,num_nodes])).to_dense()
print(adjdense.shape)

torch.Size([2120, 2120])


## 1.3 Generate graphs to calculate q() and DFS

In [ ]:
import random
from torch_geometric.utils import add_self_loops,to_networkx

edge_index, _ = add_self_loops(edge_index, num_nodes=PyTSubGraph.x.size(0))

edge_index_list = torch.transpose(edge_index, 0, 1).tolist()

node1 = [x[0] for x in edge_index_list] # out nodes
node2 = [x[1] for x in edge_index_list] # in nodes

q_1_dict, mask = load_item_pop(edge_index_list)

# set the length of DFS
walks_num = 5
candidates = candidate_choose(SubGraph, mask, walks_num)
print(candidates[0])

[1574, 1329, 69, 1178, 5]


# 2 Model Definition
## 2.1 Defining the MCGCN model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import time

class MCGCN(torch.nn.Module):
    def __init__(self,num_layers,hidden_channels,candidates,start_given,q_1_dict,N_steps,node1):
        super(MCGCN, self).__init__()
        
        self.num_layers = num_layers
        self.CONVs = torch.nn.ModuleList()
        self.MCCONVs = torch.nn.ModuleList()
        self.NegRates = nn.ParameterList()
        for layer in range(self.num_layers-1):
            if layer == 0:
                self.CONVs.append(GCNConv(dataset.num_features, hidden_channels))
                self.MCCONVs.append(GCNConv(dataset.num_features, hidden_channels))
                self.NegRates.append(nn.Parameter(torch.FloatTensor([1])))
            else:
                self.CONVs.append(GCNConv(hidden_channels, hidden_channels))
                self.MCCONVs.append(GCNConv(hidden_channels, hidden_channels))
                self.NegRates.append(nn.Parameter(torch.FloatTensor([1])))
        self.CONVs.append(GCNConv(hidden_channels, dataset.num_classes))
        self.MCCONVs.append(GCNConv(hidden_channels, dataset.num_classes))
        self.NegRates.append(nn.Parameter(torch.FloatTensor([1])))

        self.NegRate = 1
        self.candidates = candidates
        self.start_given = start_given
        self.q_1_dict = q_1_dict
        self.N_steps = N_steps
        self.node1 = node1

    def forward(self, x, edge_index,MCedge_index_list,train_neg_rate):
        for layer in range(self.num_layers):
          posi_x = self.CONVs[layer](x, edge_index)
          if len(MCedge_index_list) < self.num_layers:
            nega_sample = self.negative_sampling(posi_x,x, edge_index.shape[1]) 
            #get negative sampling node index
            nega_sample = torch.LongTensor(nega_sample)
            NegOut = edge_index[0].unsqueeze(0)
            NegIn = nega_sample.unsqueeze(0)
            NegaIndex = torch.cat((NegOut, NegIn), 0)
            MCedge_index_list.append(NegaIndex)
          else:
            NegaIndex = MCedge_index_list[layer]

          nega_x = self.MCCONVs[layer](x, NegaIndex)
          if train_neg_rate: #NegRate is a trainable parameter
            x = posi_x - self.NegRates[layer] * nega_x
          else:
            x = posi_x - self.NegRate * nega_x

          if layer < (self.num_layers-1):
            x = x.relu()
            x = F.dropout(x, p=0.5, training=self.training)

        return x,MCedge_index_list
    
    #Geting negative samplings based on Monte Carlo chains
    def negative_sampling(self,posi,x,num_edges):
        # distribution = [i/np.sum(distribution) for i in distribution]
        if self.start_given is None:
            start = np.random.choice(list(self.q_1_dict.keys()),num_edges)  # random init nodes 
        else:
            start = self.start_given
        count = 0
        cur_state = start #index of input node
        user_list = self.node1
        walks = defaultdict(list)
        generate_examples = list()
        while True:
            y_list = list()
            q_probs_list = list()
            #(1) Generate y ∼ q(y|X(t)) where the proposal distribution q is arbitrary chosen as long as positive everywhere.
            q_probs_next_list = list()
            count += 1
            sample_num = np.random.random() 
            if sample_num < 0.5: # sample proba < 0.5 negtive correlation q_probs = 0.01
                #q_1_dict : 1/deg_sum                        len(cur_state)=batch size=512   Same probability for each point
                y_list = np.random.choice(list(self.q_1_dict.keys()), len(cur_state), p=list(self.q_1_dict.values()))
                #q(i,j) Same probability for each point
                q_probs_list = [self.q_1_dict[i] for i in y_list]
                #q(j,i) Same probability for each point
                q_probs_next_list = [self.q_1_dict[i] for i in cur_state]
            else:
                for i in cur_state:
                    distribution = [1/len(self.candidates[i])] * len(self.candidates[i])
                    y = np.random.choice(self.candidates[i], 1, p=distribution)[0]
                    y_list.append(y)
                    index = self.candidates[i].index(y) # index of y in candidates[i]
                    q_probs = distribution[index]
                    q_probs_list.append(q_probs)
                    node_list_next = self.candidates[y]
                    if i in node_list_next:
                        index_next = node_list_next.index(i)
                        q_probs_next = distribution[index_next]
                    else:
                        q_probs_next = self.q_1_dict[i]
                    q_probs_next_list.append(q_probs_next) 

            q_probs_list = torch.Tensor(q_probs_list)
            q_probs_next_list = torch.Tensor(q_probs_next_list)
            user_list_value = posi[user_list]
            cur_state_value = posi[cur_state]
            y_list_value = posi[y_list]
            p_probs = torch.sigmoid(torch.pow(torch.sum(user_list_value * y_list_value, axis=1), 0.25))
            p_probs_next = torch.sigmoid(torch.pow(torch.sum(user_list_value * cur_state_value, axis=1), 0.25))
            u = np.random.rand()
            p_probs = p_probs
            p_probs_next = p_probs_next
            A_a_list = (p_probs * q_probs_next_list)/(p_probs_next * q_probs_list)
            A_a_list = A_a_list.detach().numpy()
            next_state = list()
            next_user = list()
            # N_steps = 5
            if count > self.N_steps:
                for i in list(range(len(cur_state))):
                  walks[user_list[i]].append(y_list[i])
   
            else:# count < 5
                for i in list(range(len(cur_state))):
                  A_a = A_a_list[i]
                  #   alpha                    
                  alpha = min(1, A_a)
                  if u < alpha:
                    #accept
                    next_state.append(y_list[i])
                  else:
                    next_state.append(cur_state[i])
                cur_state = next_state
            length = 0
            for key in walks.keys():
                length += len(walks[key])

            if length == num_edges:
                generate_examples = list()
                for user in node1:
                    d = walks[user]
                    if len(d) == 1:
                        generate_examples.append(d[0])    
                    else:
                        generate_examples.append(d[0])
                        del walks[user][0]
                break
            else:
                continue  
        return generate_examples

## 2.2 Defining the MAD

In [ ]:
def MAD(x):
  x_norm = x / x.norm(dim=1)[:, None]
  dist = 1-torch.mm(x_norm, x_norm.transpose(0,1))
  one1 = torch.ones_like(dist)
  zreo1 = torch.zeros_like(dist)
  dist_1 = torch.where(dist > 0, one1, zreo1)
  D = dist.sum(0)/dist_1.sum(0)
  one = torch.ones_like(D)
  zero = torch.zeros_like(D)
  D_1 = torch.where(D > 0, one, zero)
  Mad = D.sum()/D_1.sum()
  return Mad.detach().numpy()

#3 Training

In [ ]:
import sys

def test(MCedge_index_list,train_neg_rate):
      model.eval()
      out,_ = model(PyTSubGraph.x, edge_index,MCedge_index_list,train_neg_rate)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[PyTSubGraph.test_mask] == PyTSubGraph.y[PyTSubGraph.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(PyTSubGraph.test_mask.sum())  # Derive ratio of correct predictions.
      return test_acc


#----------------
# Set training parameters
num_layers = 5
Runs = 11
Epochs = 201
N_steps = 5 #Step for Monte Carlo chains
start_given = None
#-----------------
#Set whether NegRate is a trainable parameter
#if False:
#   NegRate = 1
train_neg_rate = False
#-----------------

for num in range(1,Runs):
  model = MCGCN(num_layers=num_layers,
          hidden_channels=16,
          candidates=candidates,
          start_given=start_given,
          q_1_dict=q_1_dict,
          N_steps=N_steps,
          node1 = node1)
  if num == 1 :
    print(model)
  print("num_layers",num_layers)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
  criterion = torch.nn.CrossEntropyLoss()
  for epoch in range(1, Epochs):
      start = time.time()
      MCedge_index_list = []
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out,MCedge_index_list = model(PyTSubGraph.x, edge_index,MCedge_index_list,train_neg_rate)  
      loss = criterion(out[PyTSubGraph.train_mask], PyTSubGraph.y[PyTSubGraph.train_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()
      end = time.time()
      if epoch == 1:
          print("---------------")
          print("MCedge_index_shape",MCedge_index_list[0].shape)
          print("Epoch time:",end-start)
          print("---------------")
      if epoch % 10 == 0:
          test_acc = test(MCedge_index_list,train_neg_rate)
          print(f'num: {num:03d}, Epoch: {epoch:03d}, Loss: {loss:.4f},Test Accuracy: {test_acc:.4f},MAD: {MAD(out):.4f}')


MCGCN(
  (CONVs): ModuleList(
    (0): GCNConv(
      (lin): Linear(in_features=3703, out_features=16, bias=True)
    )
    (1): GCNConv(
      (lin): Linear(in_features=16, out_features=16, bias=True)
    )
    (2): GCNConv(
      (lin): Linear(in_features=16, out_features=16, bias=True)
    )
    (3): GCNConv(
      (lin): Linear(in_features=16, out_features=16, bias=True)
    )
    (4): GCNConv(
      (lin): Linear(in_features=16, out_features=6, bias=True)
    )
  )
  (MCCONVs): ModuleList(
    (0): GCNConv(
      (lin): Linear(in_features=3703, out_features=16, bias=True)
    )
    (1): GCNConv(
      (lin): Linear(in_features=16, out_features=16, bias=True)
    )
    (2): GCNConv(
      (lin): Linear(in_features=16, out_features=16, bias=True)
    )
    (3): GCNConv(
      (lin): Linear(in_features=16, out_features=16, bias=True)
    )
    (4): GCNConv(
      (lin): Linear(in_features=16, out_features=6, bias=True)
    )
  )
  (NegRates): ParameterList(
      (0): Parameter conta